
# **Семинар и дз 6**

#### Задача 1.

Перепишите класс с пациентом, у которого есть температура, с использованием декоратора.

In [ ]:
class Patient:
  def __init__(self, diagnos, temperature):
    self._diagnos = diagnos
    self._temperature = temperature

  @property
  def temperature(self):
    return self._temperature

  @temperature.setter
  def temperature(self, value):
    if 37 <= value <= 41:
      print(f'У пациента поднялась температура!')
      self._temperature = value
    elif 34 <= value <= 36.9:
      print(f'Температура пациента в пределах нормы!')
      self._temperature = value
    else:
      print(f'Пациент находится в критическом состоянии!')
      self._temperature = value

In [ ]:
patient = Patient('Воспаление легких', 36.6)
print(patient.temperature)

36.6


In [ ]:
patient.temperature = 38.5
print(patient.temperature)

У пациента поднялась температура!
38.5


In [ ]:
patient.temperature = 42
print(patient.temperature)

Пациент находится в критическом состоянии!
42


#### Задача 2.

Напишите декоратор, который будет конвертировать результат выполнения функции в bool.

In [ ]:
from string import printable
def to_bool(func):
  def convert(*args, **kwargs):
    res = func(*args, **kwargs)
    return bool(res)
  return convert
@to_bool
def function(value):
  return value
print((function('Hello world')))

<function bool.<locals>.convert at 0x78d7bc8ac310>


#### Задача 3.

Напишите декоратор-класс, который будет пытаться вызывать функцию повторно, если во время ее выполнения возникла какая-то ошибка (представим себе, что у нас есть функции API, которые иногда возвращают ошибку при сбое соединения: тогда нужно подождать и повторно их вызвать).

In [ ]:
import time
class Function:
  def __init__(self, attempts=3, delay=1):
    self.attempts = attempts
    self.delay = delay
  def __call__(self, func):
    def argument(*args, **kwargs):
      attempts = 0
      while attempts < self.attempts:
        try:
          return func(*args, **kwargs)
        except Exception as e:
          attempts += 1
          print(f'Количество неудачных попыток: {attempts} {e}')
          if attempts < self.attempts:
            time.sleep(self.delay)
      raise Exception(f'Функция не работает {func.__name__} после {self.attempts} попыток')
    return argument

In [ ]:
@Function(attempts=5, delay=2)
def uncall():
  import random
  if random.choice([True, False]):
    raise ValueError('Случайная ошибка')
  return 'Успешно!'

In [ ]:
try:
    result = uncall()
    print(result)
except Exception as e:
    print(e)

Количество неудачных попыток: 1 Случайная ошибка
Успешно!


#### Задача 4.

Допустим, у нас есть функция, которая возвращает сумму своих аргументов. Напишите декоратор, который будет позволять такой функции складывать попарно элементы в кортежах, которые передаются списком, то есть, список [(1, 2), (3, 4)] вернет [3, 7].

In [ ]:
def decorator(func):
  def arguments(turples):
    result = [sum(pair) for pair in tuples]
    return result
  return arguments

@decorator
def pairs(turples):
  return turples
tuples = [(1,2),(3,4)]
result = pairs(tuples)
print(result)

[3, 7]


Задача 1.

С помощью модуля time реализуйте декоратор, который будет высчитывать время выполнения функции.

In [ ]:
import time
def decorator(func):
  def arguments(*args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    all_time = end_time - start_time
    print(f'Время выполнения функции {func.__name__}: {all_time:.5f} секунд')
    return result
  return arguments

@decorator
def function(n):
  total = 0
  for i in range(n):
    total += i
  return total
result = function(105000)
print(f'Результат выполнения: {result}')

Время выполнения функции function: 0.01691 секунд
Результат выполнения: 5512447500


Задача 2.

Перепишите класс Token с использованием декоратора dataclass.

In [ ]:
!pip install pymorphy2

In [ ]:
from dataclasses import dataclass
import pymorphy2
@dataclass
class Token:
  text: str
  _lemma: str = None
  pos: str = None
  dep: str = None
  ent_type: str = None
  morph = pymorphy2.MorphAnalyzer()

  @property
  def lemma(self):
    if self._lemma is None:
      self._lemma = self.morph.parse(self.text)[0].normal_form
    return self._lemma

  @lemma.setter
  def lemma(self, lemma):
    self._lemma = lemma

  def set_pos(self, pos):
    self.pos = pos
  def set_dep(self, dep):
    self.dep = dep
  def set_ent_type(self, ent_type):
    self.ent_type = ent_type

  def lemmatize(self):
    if self._lemma is None:
      self._lemma = self.morph.parse(self.text)[0].normal_form

In [ ]:
token = Token('места')
print(f'Text: {token.text}')
print(f'Lemma: {token._lemma}')
token.set_pos('NOUN')
print(f'POS: {token.pos}')
token.set_ent_type('MISC')
token.set_dep('nsubj')
print(f'Nominal subject: {token.dep}')
token.lemmatize()
print(f'Лемма после лемматизации: {token._lemma}')

Text: места
Lemma: None
POS: NOUN
Nominal subject: nsubj
Лемма после лемматизации: место


Задача 3.


Расширьте декоратор таким образом, чтобы а) элементов в кортеже могло быть сколько угодно и они суммировались все б) декоратор был рекурсивным и умел отыскивать самые вложенные кортежи/списки и складывать их элементы, например: [[1, 2], [3, 4]], [[3, 4]] = 17 При этом вложенные числа должны быть все на одном уровне (то есть, не может оказаться, чтобы в списке был и список, и число).

In [ ]:
import time
def decorator(func):
  def arguments(*args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    all_time = end_time - start_time
    print(f'Время выполнения функции {func.__name__}: {all_time:.5f} секунд')
    return result
  return arguments

def elements(data):
  total = 0
  if isinstance(data, (list, tuple)):
    for item in data:
      total += elements(item) # будет рекурсивно суммировать элементы
  elif isinstance(data, (int, float)):
      total += data
  return total

@decorator
def function(*args):
  total_sum = 0
  for arg in args:
    total_sum += elements(arg) # попробую суммировать элементы из аргументов
  return total_sum

In [ ]:
result = function(
    (1, 2, (3, 4)),
    [5, [6, 7]],
    [[8, 9], [10]],
    (11, 12)
)
print(f'Результат: {result}')

Время выполнения функции function: 0.00001 секунд
Результат: 78



# **Семинар 7**

Задача 1.

Напишите метакласс с логированием: при определении любого нового класса в файл (путь к которому хранится в статическом атрибуте метакласса) записывается имя этого класса, его суперклассы и атрибуты.

То есть правильно ли я поняла, что мне сначала нужно:
создать метакласс, затем добавить статический атрибут для того, чтобы хранился путь к файлу, а затем записать информацию в файл?

Лог (log) - это хронологическая запись наиболее значимой информации о работе системы.

Срочное исправление - Fatal error

Ошибки, не влияющие на пользователя - Not critical error

События, требующие внимания - Warning

Информация о вызовах сервера - Initial information

In [ ]:
# Пример с Geek
# importing module
import logging

# Create and configure logger
logging.basicConfig(filename="newfile.log",
                    format='%(asctime)s %(message)s',
                    filemode='w')

# Creating an object
logger = logging.getLogger()

# Setting the threshold of logger to DEBUG
logger.setLevel(logging.DEBUG)

# Test messages
logger.debug("Harmless debug Message")
logger.info("Just an information")
logger.warning("Its a Warning")
logger.error("Did you try to divide by zero")
logger.critical("Internet is down")

DEBUG:root:Harmless debug Message
INFO:root:Just an information
ERROR:root:Did you try to divide by zero
CRITICAL:root:Internet is down


In [17]:
class Meta(type):
  log_file = 'булгаков.txt'
  def __new__(cls, name, bases, attrs):
    new_class = super().__new__(cls, name, bases, attrs)
    class_info = f'Class name: {name}'
    class_info += f'Base classes: {[base.__name__ for base in bases]}'
    class_info += f'Attributes: {list(attrs.keys())}'
    with open(cls.log_file, 'a', encoding='utf-8') as log_file:
      log_file.write(class_info)
    return new_class

class Person(metaclass=Meta):
  name = 'Петя'
  surname = 'Олегров'
  age = 50

Задача 2.

Напишите метакласс, который любому новому классу будет добавлять метод sayHello(*args, **kwargs).

In [ ]:
class Meta(type):
  def __new__(cls, name, bases, attrs):
    def sayHello(self, *args, **kwargs):
      print(f'Привет от класса: {self.__class__.__name__}')
    # добавлю метод sayhello в атрибут класса
    attrs['sayHello'] = sayHello
    # теперь создам новый класс с этим методом
    new_class = super().__new__(cls, name, bases, attrs)
    return new_class

class Hello(metaclass=Meta):
  pass

class Say(metaclass=Meta):
  pass

h1 = Hello()
h2 = Say()
h1.sayHello()
h2.sayHello()

Привет от класса: Hello
Привет от класса: Say


Задача 3.

Напишите метакласс, который позволяет своим классам иметь только атрибуты name, surname, age и никаких других.

In [ ]:
class Meta(type):
  def __new__(cls, name, bases, attrs):
    # создам список атрибутов, которые будут разрешены
    attributes = {'name', 'surname', 'age'}

    for attr in attrs.keys():
      if attr not in attributes and not attr.startswith('__'):
        raise AttributeError(f'Такой атрибут: {attr} не разрешен в классе {name}')
    new_class = super().__new__(cls, name, bases, attrs)
    return new_class

In [ ]:
class Person(metaclass=Meta):
  name = 'Петя'
  surname = 'Олегров'
  age = 50

per = Person()
print(per.name, per.surname, per.age, sep='\n')

Петя
Олегров
50


In [ ]:
class ErrorPerson(metaclass=Meta):
  name = 'Василий'
  surname = 'Геннадьев'
  age = 30
  weight = 90
er = ErrorPerson()

AttributeError: Такой атрибут: weight не разрешен в классе ErrorPerson

Задача 4 (2 балла).

Приготовимся в большом проекте создать игру-стратегию (графику потом дизайнеры добавят). Наша игра будет называться Dawn of Man, и задачей игрока будет управлять поселением первобытных людей, которые в процессе набирают очки знаний и открывают новые технологии. Наши люди - это отдельные юниты, которые умеют взаимодействовать с элементами окружающей среды: у них будут какие-то инструменты для создания еды, оружия и прочего (например, какой-нибудь навес, под которым человек садится, набрав палок и камней, и ваяет себе палку-копалку).

Наша задача - набросать черновик для игры: создать абстрактные классы для людей, объектов, с которыми они взаимодействуют, чтобы создавать вещи, и собственно вещей. Возможно, имеет смысл также предусмотреть черновик-класс для животных, на которых люди будут охотиться.

Попробуйте также создать конкретный класс для кого-нибудь из них.

Абстрактный класс - это своего рода шаблон для классов, которые будут от него наследовать. Основное отличие абстрактного класса (и абстрактных методов) в том, что нельзя создавать экземпляры этого класса (и с такими методами)

In [ ]:
from abc import ABC, abstractmethod
# не создавала конкретный класс для них, просто пока подумала, как может выглядеть абстрактный класс

class People(ABC):
  def __init__(self, name, health, knowledge):
    self.name = name
    self.health = health
    self.knowledge = knowledge

  @abstractmethod
  def pick_resources(self): # будут собирать ресурсы
    pass

  @abstractmethod
  def craft_smth(self): # создание вещей
    pass
# Люди будут взаимодействовать с ресурсами
class Resources(ABC):
  @abstractmethod
  def collect(self):
    pass
# Люди могут использовать собранные предметы
class Things(ABC):
  @abstractmethod
  def use(self):
    pass

#Животные, которые будут охотиться
class Animal(ABC):
  def __init__(self, species, health):
    self.species = species
    self.health = health
  @abstractmethod
  def hunt(self):
    pass